<a href="https://colab.research.google.com/github/stutibimali/Webscrapping/blob/main/BritishAirlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA COLLECTION

In [1]:
#import
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#creating empty list

reviews=[]
stars=[]
date=[]
country=[]

In [3]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100


# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    #reviews
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    #stars
    for para in parsed_content.find_all("div", {"class": "rating-10"}):
      try:
          stars.append(para.span.get_text())
      except:
          print(f"Error on page {i}")
          stars.append("None")
    #country
    for para in parsed_content.find_all("h3",{"class":"text_sub_header userStatusWrapper"}):
        a=para.get_text().strip()
        last= a.rfind('(')
        first=a.find(')', last)
        country.append(a[last + 1 : first])
    #date
    for para in parsed_content.find_all("time"):
        date.append(para.text)
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [4]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,✅ Trip Verified | The incoming and outgoing f...
1,✅ Trip Verified | Back in December my family ...
2,✅ Trip Verified | As usual the flight is dela...
3,✅ Trip Verified | A short BA euro trip and thi...
4,Not Verified | We are flying Business class f...


In [5]:
df = pd.DataFrame()
df["stars"] = stars
df.head()

,stars
0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5
1,1
2,1
3,1
4,8


In [6]:
df = pd.DataFrame()
df["country"] = country
df.head()

,country
0,United Kingdom
1,Australia
2,United Kingdom
3,United Kingdom
4,United States


In [7]:
df = pd.DataFrame()
df["date"] = date
df.head()

,date
0,10th March 2023
1,10th March 2023
2,10th March 2023
3,9th March 2023
4,8th March 2023


In [8]:
len(reviews)

1000

In [9]:
len(country)

1000

In [10]:
stars = stars[:1000]

In [11]:
# create a data frame to store all

df = pd.DataFrame({"reviews":reviews,"stars": stars, "date":date, "country": country})
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | The incoming and outgoing f...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,10th March 2023,United Kingdom
1,✅ Trip Verified | Back in December my family ...,1,10th March 2023,Australia
2,✅ Trip Verified | As usual the flight is dela...,1,10th March 2023,United Kingdom
3,✅ Trip Verified | A short BA euro trip and thi...,1,9th March 2023,United Kingdom
4,Not Verified | We are flying Business class f...,8,8th March 2023,United States


In [12]:
df.shape

(1000, 4)

In [13]:
import os
cwd = os.getcwd()
df.to_csv(cwd+ "/BA_reviews.csv")

# Data Cleaning

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import os


In [15]:
#create a dataframe from csv file

cwd = os.getcwd()

df = pd.read_csv(cwd+"/BA_reviews.csv", index_col=0)
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | The incoming and outgoing f...,5,10th March 2023,United Kingdom
1,✅ Trip Verified | Back in December my family ...,1,10th March 2023,Australia
2,✅ Trip Verified | As usual the flight is dela...,1,10th March 2023,United Kingdom
3,✅ Trip Verified | A short BA euro trip and thi...,1,9th March 2023,United Kingdom
4,Not Verified | We are flying Business class f...,8,8th March 2023,United States


In [16]:
df['verified'] = df.reviews.str.contains("Trip Verified")
df['verified']

0       True
1       True
2       True
3       True
4      False
       ...  
995     True
996     True
997     True
998     True
999     True
Name: verified, Length: 1000, dtype: bool

## Cleaning Reviews

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [20]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [21]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [22]:
df.head()

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | The incoming and outgoing f...,5,10th March 2023,United Kingdom,True,incoming outgoing flight delayed french air tr...
1,✅ Trip Verified | Back in December my family ...,1,10th March 2023,Australia,True,back december family getting onto plane refuse...
2,✅ Trip Verified | As usual the flight is dela...,1,10th March 2023,United Kingdom,True,usual flight delayed week already hour held bu...
3,✅ Trip Verified | A short BA euro trip and thi...,1,9th March 2023,United Kingdom,True,short ba euro trip ba excel clean aircraft goo...
4,Not Verified | We are flying Business class f...,8,8th March 2023,United States,False,verified flying business class flight premium ...


## Format Date

In [23]:
df.dtypes

reviews     object
stars        int64
date        object
country     object
verified      bool
corpus      object
dtype: object

In [24]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

In [25]:
df.date.head()

0   2023-03-10
1   2023-03-10
2   2023-03-10
3   2023-03-09
4   2023-03-08
Name: date, dtype: datetime64[ns]

## Cleaning star

In [26]:
#check for unique values
df.stars.unique()

array([ 5,  1,  8,  2, 10,  4,  6,  7,  9,  3])

In [28]:
df.stars.value_counts()

1     305
2     111
10    104
3      96
8      80
9      80
5      61
4      60
7      60
6      43
Name: stars, dtype: int64

## Check for null

In [29]:
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     1000
dtype: int64

In [30]:
df.country.isnull().value_counts()

False    1000
Name: country, dtype: int64

In [31]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [32]:
df.shape

(1000, 6)

In [33]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | The incoming and outgoing f...,5,2023-03-10,United Kingdom,True,incoming outgoing flight delayed french air tr...
1,✅ Trip Verified | Back in December my family ...,1,2023-03-10,Australia,True,back december family getting onto plane refuse...
2,✅ Trip Verified | As usual the flight is dela...,1,2023-03-10,United Kingdom,True,usual flight delayed week already hour held bu...
3,✅ Trip Verified | A short BA euro trip and thi...,1,2023-03-09,United Kingdom,True,short ba euro trip ba excel clean aircraft goo...
4,Not Verified | We are flying Business class f...,8,2023-03-08,United States,False,verified flying business class flight premium ...
...,...,...,...,...,...,...
995,✅ Trip Verified | This was an early morning f...,10,2018-05-18,Singapore,True,early morning flight heathrow keflavik flight ...
996,✅ Trip Verified | Cape Town to London. Dated ...,9,2018-05-17,United Kingdom,True,cape town london dated aircraft poor cabin ser...
997,✅ Trip Verified | Washington to Prague via Lo...,1,2018-05-14,United States,True,washington prague via london flown several tim...
998,✅ Trip Verified | London to Washington. Apart...,1,2018-05-14,France,True,london washington apart space nothing nothing ...


In [34]:
# export the cleaned data

df.to_csv(cwd + "/cleaned-BA-reviews.csv")